In [ ]:
! pip install scikit-commpy

In [ ]:
#Data
import math
import numpy as np
import commpy.channelcoding.convcode as cc
import commpy.modulation as modulation
import numpy as np

N= 512 #Number of modulated signal subcarriers in one time slot #########if N>63 Hermision not work why?????????????
L=3 #number of modulated signal time slots
NN= (N*2)+2 #total Number of subcarriers in one time slot after Hermision
LL=3 #total number of time slots after Hermision
M = 4 # modulation order (QPSK)
K = int(np.log2(M)) #number of bit per modulation symbol
Data1 = np.random.randint(2,size=(K*N*L))
print(Data1)
print(Data1.shape)

[0 0 0 ... 0 1 0]
(3072,)


In [ ]:
#M = 4 # modulation order (QPSK)
#K = np.log2(M) #number of bit per modulation symbol
modem = modulation.PSKModem(M) # M-PSK modem initialization
modulated_uncoded2 = modem.modulate(Data1) 
#modulated = modem.modulate(Data1) # modulation
print(modulated_uncoded2)
print(modulated_uncoded2.shape)
#print(modulated[2])

[ 1.0000000e+00+0.j  6.1232340e-17+1.j  6.1232340e-17+1.j ...
 -1.8369702e-16-1.j  1.0000000e+00+0.j -1.8369702e-16-1.j]
(1536,)


In [ ]:
modulated_uncoded= np.zeros((N,L), dtype=complex)
i=0
k=0
t=0
while i<L:
      k=0
      while k<N:
            g=modulated_uncoded2[t]
            modulated_uncoded[k,i]=g
            t+=1
            k+=1
      i+=1
print(modulated_uncoded)

[[ 1.0000000e+00+0.0000000e+00j  1.0000000e+00+0.0000000e+00j
   6.1232340e-17+1.0000000e+00j]
 [ 6.1232340e-17+1.0000000e+00j  1.0000000e+00+0.0000000e+00j
   1.0000000e+00+0.0000000e+00j]
 [ 6.1232340e-17+1.0000000e+00j -1.0000000e+00+1.2246468e-16j
   1.0000000e+00+0.0000000e+00j]
 ...
 [ 6.1232340e-17+1.0000000e+00j  1.0000000e+00+0.0000000e+00j
  -1.8369702e-16-1.0000000e+00j]
 [-1.0000000e+00+1.2246468e-16j  1.0000000e+00+0.0000000e+00j
   1.0000000e+00+0.0000000e+00j]
 [-1.0000000e+00+1.2246468e-16j  6.1232340e-17+1.0000000e+00j
  -1.8369702e-16-1.0000000e+00j]]


In [ ]:
def Hermitian(matrix):
    conjec= np.zeros((N,L), dtype=complex)
    i=0
    while i<L:
          conjec[:,i]=np.conj(matrix[:,i])
          i+=1
    reversed_conjec=conjec[-1::-1]
    z = np.zeros((1,L)) 
    modulated_uncodeda=np.vstack(z)
    modulated_uncodeda=np.vstack((modulated_uncodeda,matrix))
    modulated_uncodedb=np.vstack((modulated_uncodeda,z))
    Hermitian_Data=np.vstack((modulated_uncodedb,reversed_conjec))
    return Hermitian_Data

In [ ]:
Hermitian_Data=Hermitian(modulated_uncoded)

In [ ]:
print(Hermitian_Data)

[[ 0.000000e+00+0.0000000e+00j  0.000000e+00+0.0000000e+00j
   0.000000e+00+0.0000000e+00j]
 [ 1.000000e+00+0.0000000e+00j  1.000000e+00+0.0000000e+00j
   6.123234e-17+1.0000000e+00j]
 [ 6.123234e-17+1.0000000e+00j  1.000000e+00+0.0000000e+00j
   1.000000e+00+0.0000000e+00j]
 ...
 [ 6.123234e-17-1.0000000e+00j -1.000000e+00-1.2246468e-16j
   1.000000e+00-0.0000000e+00j]
 [ 6.123234e-17-1.0000000e+00j  1.000000e+00-0.0000000e+00j
   1.000000e+00-0.0000000e+00j]
 [ 1.000000e+00-0.0000000e+00j  1.000000e+00-0.0000000e+00j
   6.123234e-17-1.0000000e+00j]]


In [ ]:
#IFFT
p=0
Data_ifft = np.zeros((NN,L), dtype=complex)  #per column
while p<L:
      Data_ifft[:,p]= np.fft.ifft(Hermitian_Data[:,p])
      p+=1
print(Data_ifft)
print(Data_ifft.shape)
#Data_ifft = np.fft.ifft(Data)
data_final = Data_ifft

[[-0.00584795-8.65670974e-19j  0.00974659-4.32835487e-19j
   0.04288499+3.24626615e-19j]
 [-0.00010408+4.32835487e-19j  0.00765176+3.24626615e-18j
  -0.020523  -1.94775969e-18j]
 [ 0.03310869+9.36006741e-18j -0.0211598 -3.13805728e-18j
  -0.01348327+1.08208872e-18j]
 ...
 [-0.02606224-7.38837617e-18j -0.02028185-2.75469919e-18j
   0.01966314-6.38345413e-18j]
 [ 0.04475389-8.72080097e-18j -0.03257978+4.35651782e-18j
   0.00979289-3.49302685e-18j]
 [-0.00288981+4.93559834e-18j -0.02788105-2.21475084e-18j
  -0.03056395-9.02488032e-18j]]
(1026, 3)


In [ ]:
#CP
Channel=[1,0.5,0.25]
#length of cycle prefix must be >= length of channel
CP = Data_ifft[NN-3:NN,:]
#print(CP)
Data_CP = np.vstack((CP, Data_ifft))
print(Data_CP)
data_final = Data_CP
print(data_final.shape)

[[-0.02606224-7.38837617e-18j -0.02028185-2.75469919e-18j
   0.01966314-6.38345413e-18j]
 [ 0.04475389-8.72080097e-18j -0.03257978+4.35651782e-18j
   0.00979289-3.49302685e-18j]
 [-0.00288981+4.93559834e-18j -0.02788105-2.21475084e-18j
  -0.03056395-9.02488032e-18j]
 ...
 [-0.02606224-7.38837617e-18j -0.02028185-2.75469919e-18j
   0.01966314-6.38345413e-18j]
 [ 0.04475389-8.72080097e-18j -0.03257978+4.35651782e-18j
   0.00979289-3.49302685e-18j]
 [-0.00288981+4.93559834e-18j -0.02788105-2.21475084e-18j
  -0.03056395-9.02488032e-18j]]
(1029, 3)


In [ ]:
#P/S with CP
i = 0
s = 0
t = 0
serial_data = np.zeros(((NN+3)*L), dtype=complex)
while i < L:
      k=0
      while k < NN+3: 
            g=(data_final[k,i])
            serial_data[t]=g
            t+=1
            k+=1
      i+=1    
print(serial_data)
print(serial_data.shape)

[-0.02606224-7.38837617e-18j  0.04475389-8.72080097e-18j
 -0.00288981+4.93559834e-18j ...  0.01966314-6.38345413e-18j
  0.00979289-3.49302685e-18j -0.03056395-9.02488032e-18j]
(3087,)


In [ ]:
#convolution between data and channel
from scipy.ndimage import convolve
data_rx_H = np.convolve(serial_data,Channel)
print(data_rx_H)
print(data_rx_H.shape)

[-0.02606224-7.38837617e-18j  0.03172277-1.24149890e-17j
  0.01297157-1.27189618e-18j ... -0.02075172-1.23672573e-17j
 -0.01283375-5.38569687e-18j -0.00764099-2.25622008e-18j]
(3089,)


In [ ]:
#noise
EbNo = 9 # energy per bit to noise power spectral density ratio (in dB)
#rate=1   #rate =1 as signal is uncoded
#snrdB = EbNo + 10*np.log10(2*rate)+10*np.log10(N*L)+10*np.log10()   #2=K=np.log2(M), Signal-to-Noise ratio per symbol (in dB) 
Es = np.mean(np.abs(modulated_uncoded)**2)   # symbol energy
No = Es/((10**(EbNo/10))*np.log2(M))   # noise spectrum density                                                                                 
data_rx = data_rx_H + np.sqrt(No/2)*(np.random.randn(data_rx_H.shape[0]))#+1j*np.random.randn(data_rx_H.shape[0]))   # AWGN (uncoded case)                                                                                
print(data_rx)                                                                                  
print(data_rx.shape)    

#Es/No(dB)=Eb/No(dB)+10log10(Nc*log2(M))+10log10(CodeRate)+10log10(Tinf/(Tinf+Tcp))
#M - number of symbols used in the modulation scheme
#Nc - number of subcarriers in the ofdm symbol
#CodeRate - well..code rate :)
#Tinf - information period in the ofdm system
#Tcp - cyclic prefix period in the ofdm system
#(Tcp+Tinf = Tsymbol)  ?????
#https://www.edaboard.com/threads/snr-to-ebno-in-ofdm-system.125546/


[0.17415318-7.38837617e-18j 0.37400346-1.24149890e-17j
 0.16169155-1.27189618e-18j ... 0.12097849-1.23672573e-17j
 0.08837059-5.38569687e-18j 0.22643988-2.25622008e-18j]
(3089,)


In [ ]:
#reshaping data with CP
data_rx1 = data_rx[0:12000000]
#print(data_rx1)
i = 0
s = 0
t=0
reshaped_data = np.zeros(((NN+3),L), dtype=complex)
while i < L:
      k=0
      while k < NN+3: #19
            g=(data_rx1[t])
            reshaped_data[k,i]=g
            t+=1
            k+=1
      i+=1    
print(reshaped_data)
print(reshaped_data.shape)

[[ 1.74153181e-01-7.38837617e-18j -4.29416256e-02-2.46710026e-18j
  -1.66583277e-01-6.40170010e-18j]
 [ 3.74003456e-01-1.24149890e-17j -2.61264006e-01+4.21306780e-18j
   1.41696072e-02-7.23844162e-18j]
 [ 1.61691551e-01-1.27189618e-18j -2.06678983e-01-7.25166731e-19j
  -1.09149387e-01-1.23672573e-17j]
 ...
 [ 3.13082832e-02-2.47243095e-18j -4.96328809e-04-1.25230378e-18j
   1.22700571e-01-7.11674984e-18j]
 [ 6.81216700e-02-1.02563799e-17j -1.09970838e-01+3.75470856e-18j
  -2.50913662e-01-7.18101289e-18j]
 [ 3.70466671e-04-1.27189618e-18j -1.07139635e-02-7.25166731e-19j
   1.20978495e-01-1.23672573e-17j]]
(1029, 3)


In [ ]:
#Removing cycle Prefix
data_without_CP = reshaped_data[3:NN+3,:]
print(data_without_CP)

[[ 0.0903298 -5.78072046e-19j -0.16039895-4.51081454e-19j
  -0.04072592-5.06107026e-18j]
 [ 0.17120616+1.23389959e-18j  0.09567921+2.47616070e-18j
   0.20884898-4.04166646e-18j]
 [ 0.10011641+9.36006741e-18j  0.02268943-1.62313308e-18j
   0.0351124 +1.89365526e-19j]
 ...
 [ 0.03130828-2.47243095e-18j -0.00049633-1.25230378e-18j
   0.12270057-7.11674984e-18j]
 [ 0.06812167-1.02563799e-17j -0.10997084+3.75470856e-18j
  -0.25091366-7.18101289e-18j]
 [ 0.00037047-1.27189618e-18j -0.01071396-7.25166731e-19j
   0.12097849-1.23672573e-17j]]


In [ ]:
#FFT
import scipy as sp
data_without_CP_or_fft = np.zeros((NN,L), dtype=complex)
p=0
while p<L:
      data_without_CP_or_fft[:,p]= np.fft.fft(data_without_CP[:,p])
      p+=1
print(data_without_CP_or_fft)
print(data_without_CP_or_fft.shape)

[[ 1.58585753+9.86076132e-32j -2.25994279+4.93038066e-32j
   5.36205445+2.77333912e-32j]
 [ 2.62489433+5.74826643e+00j -1.7592757 +4.05212610e+00j
  -1.84667863+1.05734886e+00j]
 [-1.02554871+2.28819107e-01j -0.16802336-9.25078823e-01j
   2.82495563-2.43239315e+00j]
 ...
 [-0.51036956-6.62542456e+00j -0.20965712+2.62535509e+00j
  -1.37986092-5.65664278e-01j]
 [-1.02554871-2.28819107e-01j -0.16802336+9.25078823e-01j
   2.82495563+2.43239315e+00j]
 [ 2.62489433-5.74826643e+00j -1.7592757 -4.05212610e+00j
  -1.84667863-1.05734886e+00j]]
(1026, 3)


In [ ]:
#channel equalyzier
import numpy as np
from numpy import matrix
from numpy import linalg
data_recover = np.zeros((NN,L), dtype=complex)
H_transpose = np.zeros((NN,1), dtype=complex)
H= np.fft.fft(Channel,NN)
#H_transpose=H.transpose()
k=0
while k < NN:
      g=(H[k])
      H_transpose[k,0]=g
      k+=1
i=0
while i<L:
      #data_recover[:,i]=np.divide(data_without_CP_or_fft[:,i],H_real)
      data_recover[:,i]=data_without_CP_or_fft[:,i]/H
      i+=1
print(data_recover)
#print(data_recover.shape)

[[ 0.9062043 +5.63472075e-32j -1.29139588+2.81736038e-32j
   3.06403112+1.58476521e-32j]
 [ 1.48845072+3.28998516e+00j -1.01340716+2.31199152e+00j
  -1.05736332+6.00508914e-01j]
 [-0.58695194+1.26654207e-01j -0.09231507-5.29296553e-01j
   1.62401316-1.37866212e+00j]
 ...
 [-0.33139713-3.78302548e+00j -0.10405813+1.50151236e+00j
  -0.79191266-3.14969783e-01j]
 [-0.58695194-1.26654207e-01j -0.09231507+5.29296553e-01j
   1.62401316+1.37866212e+00j]
 [ 1.48845072-3.28998516e+00j -1.01340716-2.31199152e+00j
  -1.05736332-6.00508914e-01j]]


In [ ]:
#P/S 
i = 0
s = 0
t = 0
serial_data = np.zeros((NN*L), dtype=complex)
while i < L:
      k=0
      while k < NN: 
            g=(data_recover[k,i])
            serial_data[t]=g
            t+=1
            k+=1
      i+=1    
print(serial_data)
print(serial_data.shape)

[ 0.9062043 +5.63472075e-32j  1.48845072+3.28998516e+00j
 -0.58695194+1.26654207e-01j ... -0.79191266-3.14969783e-01j
  1.62401316+1.37866212e+00j -1.05736332-6.00508914e-01j]
(3078,)


In [ ]:
demodulated_uncoded = modem.demodulate(serial_data, demod_type='hard') # demodulation (uncoded case)
print(demodulated_uncoded)

[0 0 0 ... 0 1 1]


In [ ]:
recovered_data=demodulated_uncoded[K:K+(K*N)]
i=1
p=3
while i<L: 
      recovered_data=np.hstack((recovered_data,demodulated_uncoded[(p*K)+((p-1)*(K*N)):(p*K)+((p-1)*(K*N))+(K*N)]))
      i+=1
      p+=2
print(recovered_data)
print(recovered_data.shape)

[0 1 1 ... 1 1 1]
(3072,)


In [ ]:
#BER_calculation
N_c = K*N*L # number of bits
i=0
BER_uncoded = np.zeros(N_c)
def BER_calc(a, b):      #func_compare_bet_the_original_and_transmitted_data
    ber = a-b                     #if bits are equal return 0 else return 1 
    while ber==0:
          return 0
    else:
         return 1
while i<N_c:
      BER_uncoded[i] = BER_calc(Data1[i], recovered_data[i]) # bit-error ratio 
      i+=1
print(BER_uncoded)

mean_BER_uncoded = BER_uncoded.mean() # averaged bit-error ratio (uncoded case)
print('BER =',mean_BER_uncoded)
similarity=1-mean_BER_uncoded #the similarity between the recieved and original signal
print('similarity between the recieved and original signal =',similarity*100, '%')

[0. 1. 1. ... 1. 0. 1.]
BER = 0.4462890625
similarity between the recieved and original signal = 55.37109375 %


In [ ]:
#Hermitian matrix
Nofcolumns=math.ceil(N*L/18)
Nofsymbols=N*L
if Nofcolumns > 1:
   conjec=np.conj(modulated_uncoded2)
   #reversed_conjec=conjec[-1::-1]
   #print(reversed_conjec)                                
   z = np.zeros(1) 
   modulated_uncodeda=np.hstack((z))
   modulated_uncodeda=np.hstack((modulated_uncodeda,modulated_uncoded2[0:18]))      #0:17
   modulated_uncodedb=np.hstack((modulated_uncodeda,z))
   conjec=np.conj(modulated_uncoded2[0:18])
   reversed_conjec1=conjec[-1::-1]
   modulated_uncodedc=np.hstack((modulated_uncodedb,reversed_conjec1[0:18]))        #0:17
   #modulated_uncodedc=np.hstack((modulated_uncodedb,conjec))
   i=2
   while i < Nofcolumns:
         modulated_uncodedc=np.hstack((modulated_uncodedc,z))
         modulated_uncodedc=np.hstack((modulated_uncodedc,modulated_uncoded2[(18*(i-1)):((18*i))]))      #(18*i)-1
         conjec=np.conj(modulated_uncoded2[(18*(i-1)):((18*i))])
         reversed_conjec=conjec[-1::-1]
         modulated_uncodedc=np.hstack((modulated_uncodedc,z,reversed_conjec))       #(18*i)-1
         i+=1
   while i == Nofcolumns: 
         conjec=np.conj(modulated_uncoded2[18*(i-1):])
         reversed_conjec=conjec[-1::-1]
         modulated_uncodedc=np.hstack((modulated_uncodedc,z,modulated_uncoded2[18*(i-1):],z,reversed_conjec))
         i+=1

   print(modulated_uncodedc)

if Nofcolumns <= 1:
   conjec=np.conj(modulated_uncoded2)
   reversed_conjec=conjec[-1::-1]
   z = np.zeros(1) 
   modulated_uncodeda=np.hstack((z))
   modulated_uncodeda=np.hstack((modulated_uncodeda,modulated_uncoded2))
   modulated_uncodedb=np.hstack((modulated_uncodeda,z))
   #modulated_uncodedc=np.hstack((modulated_uncodedb,conjec))
   modulated_uncodedc=np.hstack((modulated_uncodedb,reversed_conjec))
   #modulated_uncodedc=np.hstack((modulated_uncodedc,z))
   print(modulated_uncodedc)
print(modulated_uncodedc.shape)

[ 0.000000e+00+0.0000000e+00j  1.000000e+00+0.0000000e+00j
  6.123234e-17+1.0000000e+00j ...  6.123234e-17-1.0000000e+00j
  6.123234e-17-1.0000000e+00j -1.000000e+00-1.2246468e-16j]
(3244,)
